# Easy Scraping

Demo: scrapely, python-readability, pyQuery, httpie

Prerequisites:

* Python3
* `pip install -r reuiqrements.txt`

Useful trick in IPython notebook

In [48]:
from IPython.core.display import HTML

In [61]:
HTML('Logo of Initium Lab: <img src="%s">' % 'http://initiumlab.com/favicon-32x32.png')

A small hack to allow longer output area

In [58]:
%%javascript
//IPython.OutputArea.auto_scroll_threshold = 9999;
IPython.OutputArea.prototype._should_scroll = function(){return false;}

<IPython.core.display.Javascript object>

## Readability

We use a version ported to Python3:
<https://github.com/hyperlinkapp/python-readability>
(already included in the `reuqirements.txt` file)

In [23]:
from readability.readability import Document
import requests
html = requests.get('http://initiumlab.com/').content
readable_article = Document(html).summary()
readable_title = Document(html).short_title()

In [24]:
print(readable_article)

<html><body><div><div class="post-body">

      
      

      
        
          <video controls="" poster="./blog/20150922-jackathon3-review/jackathon3-timelapse-poster.png"><br/>  <source src="./blog/20150922-jackathon3-review/jackathon3-timelapse.mp4" type="video/mp4"/><br/>  <source src="./blog/20150922-jackathon3-review/jackathon3-timelapse.webm" type="video/webm"/><br/>  Sorry, you browser does not support HTML5 video.<br/></video>

<p>The video is also available on <a href="https://youtu.be/zFeSh2W1_C8">YouTube</a> and <a href="http://v.youku.com/v_show/id_XMTM0MzM1MjEwMA==.html?from=y1.7-2">Youku</a>.</p>
<h2 id="What_did_we_do?">What did we do?</h2><p>Jackathon is short for “Journalism-Hackathon”. At Initium Lab, we aim to push limits of Journalism with Technology. We hold regular Jackathons to advance our knowledge and skills in using new technology for obtaining data, analysing information, and reporting.</p>
<p>This week, the goal for each participant is to read one the t

In [25]:
HTML(readable_article)

## PyQuery

Let's fix the above URL problems

In [36]:
import pyquery
r = pyquery.PyQuery(readable_article)
r('p')

[<p>, <p>, <p>]

In [38]:
r('video').attr('poster')

'./blog/20150922-jackathon3-review/jackathon3-timelapse-poster.png'

In [40]:
r('video source').attr('src')

'./blog/20150922-jackathon3-review/jackathon3-timelapse.mp4'

In [41]:
r('video').attr('poster', 'http://initiumlab.com/%s' % r('video').attr('poster'))

[<video>]

In [42]:
r('video').attr('poster')

'http://initiumlab.com/./blog/20150922-jackathon3-review/jackathon3-timelapse-poster.png'

In [43]:
r('video source').attr('src', 'http://initiumlab.com/%s' % r('video source').attr('src'))

[<source>, <source>]

In [44]:
r('video source').attr('src')

'http://initiumlab.com/./blog/20150922-jackathon3-review/jackathon3-timelapse.mp4'

In [45]:
r.html()

'<body><div><div class="post-body">\n\n      \n      \n\n      \n        \n          <video controls="" poster="http://initiumlab.com/./blog/20150922-jackathon3-review/jackathon3-timelapse-poster.png"><br/>  <source src="http://initiumlab.com/./blog/20150922-jackathon3-review/jackathon3-timelapse.mp4" type="video/mp4"/><br/>  <source src="http://initiumlab.com/./blog/20150922-jackathon3-review/jackathon3-timelapse.mp4" type="video/webm"/><br/>  Sorry, you browser does not support HTML5 video.<br/></video>\n\n<p>The video is also available on <a href="https://youtu.be/zFeSh2W1_C8">YouTube</a> and <a href="http://v.youku.com/v_show/id_XMTM0MzM1MjEwMA==.html?from=y1.7-2">Youku</a>.</p>\n<h2 id="What_did_we_do?">What did we do?</h2><p>Jackathon is short for “Journalism-Hackathon”. At Initium Lab, we aim to push limits of Journalism with Technology. We hold regular Jackathons to advance our knowledge and skills in using new technology for obtaining data, analysing information, and reporting

In [54]:
%%javascript
//IPython.OutputArea.auto_scroll_threshold = 9999;
IPython.OutputArea.prototype._should_scroll = function(){return false;}

<IPython.core.display.Javascript object>

In [57]:
HTML(r.html())

In [26]:
#html.content